In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from multiprocessing import Pool
import time
from tqdm.notebook import tqdm
from utils import format_node_names

In [2]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.precision = 4 # show 4 digits precision
folder_path_slurm_data = Path('/projects/2/prjs1098/system_analytics_2024/slurm_data')
folder_path_saving_results = Path('./results')


folder_paths_system_states = [folder_path_slurm_data / path 
                              for path in ['system_states', 'system_states_int4', 'system_states_int5']]

_ = [sorted(list(path.glob("*.txt")))
                            for path in folder_paths_system_states]
all_files = [file for folder in _ for file in folder ]
all_files

[PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_1.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_10.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_11.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_12.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_13.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_14.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_15.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_16.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/system_states/system_states_2.txt'),
 PosixPath('/projects/2/prjs1098/system_analytics_2024/slurm_data/

In [3]:

# for file_path in files:
def get_date_for_a_day(file_path):
    dict_list = []
    # open the text file
    with file_path.open(mode='r') as f:
        f_text = f.read()
    # get the data in the text file in the form of a list, each element is a text that has the sinfo result in it
    sinfo_day = f_text.split('############################')
    for sinfo_sample in sinfo_day[1:]:
        # split the sinfo result based on new lines
        sinfo_sample = sinfo_sample.split('\n')
        # the fisrt line gives us the time
        time = sinfo_sample[1]
        # go through other lines and find the information that you need
        for sample_row in sinfo_sample[3:-1]:
            # the 7th column in a line is node number, we need this to repaeat the state for the node
            node_number = int(sample_row.split()[7])
            # the state is the 8th column
            state = sample_row.split()[8]
            # the last column is the node names
            node_names = sample_row.split()[-1]
            # we need to process node names, because they are in a starnge shape, we do so and put all the data in a dictionary
            d1 = {'node':format_node_names(node_names).split(','), 'time': [time] * node_number, 'state': [state]*node_number}
            # append it and go back and do this for the next line, if this was the last line then do it for the next element in the list.
            dict_list.append(d1)
            
    # turn this each dictionary to a data frame
    df_list = [pd.DataFrame(d1) for d1 in dict_list]
    # concatenate them
    df = pd.concat(df_list, axis=0)
    # turn the time into pandas date time
    df['time'] = pd.to_datetime(df['time'])
    df.sort_values(['node', 'time'], inplace=True)
    return df
 
 


In [4]:
start_time = time.time()
with Pool(30) as pool:  
    parallel_results = pool.map(get_date_for_a_day, all_files)  # Submit tasks
parallel_duration = time.time() - start_time

In [5]:
df = pd.concat(parallel_results, axis=0)
df.sort_values(['node', 'time'], inplace=True)
display(df.head(), len(df))



,node,time,state
0,fcn1,2024-11-04 17:03:13,mixed
0,fcn1,2024-11-04 17:03:13,mixed
0,fcn1,2024-11-04 17:03:43,mixed
0,fcn1,2024-11-04 17:03:43,mixed
0,fcn1,2024-11-04 17:04:13,mixed


98292420

In [6]:
print(f"Numebr of duplicated rows based on node and time and state: {df.duplicated(['node', 'time', 'state']).sum()}")
# why is there duplication? It could be the way I am getting data?
# or sinfo gives copy of the nodes in any case some of the 
# node and time are the same. 
# I think sinfo gives the states for some nodes twice in different partitions.

# we drop the duplicated rows:
df.drop_duplicates(subset=['node', 'time', 'state'], inplace=True)

Numebr of duplicated rows based on node and time and state: 37410997


In [7]:
"""  
Here we check for duplicaton in node and time. This is extremely rare and it means that
a node at a specific time can have two states! This can happen because we are measuring the states with 1 second precision
and through 3 login nodes. 
Is there any?
"""
print(f"Numebr of duplicated rows based on node and time: {df.duplicated(['node', 'time']).sum()}")

# show a sample
df[df.duplicated(['node', 'time'], keep=False)].head(n=10) 




Numebr of duplicated rows based on node and time: 131


,node,time,state
1,gcn117,2024-11-15 02:59:39,completing
0,gcn117,2024-11-15 02:59:39,draining
0,gcn14,2024-11-16 15:02:04,completing
4,gcn14,2024-11-16 15:02:04,mixed
0,gcn2,2024-11-13 23:55:10,mixed
0,gcn2,2024-11-13 23:55:10,completing
0,gcn2,2024-11-14 13:54:28,allocated
0,gcn2,2024-11-14 13:54:28,completing
0,gcn2,2024-11-14 15:40:38,allocated
0,gcn2,2024-11-14 15:40:38,completing


In [8]:
""" 
It is not clear what to do with these samples. There are only a few instances.
We keep the first occurence and delete others.
"""
df.drop_duplicates(subset=['node', 'time'], inplace=True)

# give a description of the collected data. How many samples, how many states, nodes, minimum time maximum time ... 
display(df.describe(include='all'), df['state'].value_counts())

,node,time,state
count,60881292,60881292,60881292
unique,1548,NaN,22
top,tcn999,NaN,allocated
freq,39329,NaN,24633550
mean,NaN,2024-11-12 19:14:28.834247424,NaN
min,NaN,2024-11-04 17:03:13,NaN
25%,NaN,2024-11-08 03:27:43,NaN
50%,NaN,2024-11-12 00:09:50,NaN
75%,NaN,2024-11-17 06:49:09,NaN
max,NaN,2024-11-23 07:34:37,NaN


state
allocated      24633550
idle           24502310
mixed           9505566
reserved        1171058
planned          508891
drained          298389
completing        90216
draining@         79634
draining          37803
drained*          33219
down*              9672
idle*              2841
unknown            2546
down               2198
inval              1700
reboot^            1145
mixed*              328
allocated*          120
completing*          50
mixed-               27
draining*            16
reboot               13
Name: count, dtype: int64

In [9]:
""" 
Are we measuring the data regularly? No
Here we compute the time difference for our measeruments node specific. 

state(node=node1, time=t2) - state(node=node1, time=t1) = Delta t 


what is the max and min for Delta t?
Is this acceptable? or no it makes the analysis difficult?
"""

display(df.groupby(['node'], as_index=False)[['time']].diff().describe(percentiles=[0.25, 0.5, 0.75, 0.9995]))

""" 
The statistic for Delta t shows that a large portion of the measurements have time differnce less than 3 minutes.
But it seems that at some point in our measurment process we did not record states for some nodes for around 2 hours.
This could our measurement faults or even the case that a specific nodes did not appear in the sinfo.
What are those nodes? are they srv nodes?==> No idea
"""

,time
count,60879744
mean,0 days 00:00:40.873779495
std,0 days 00:00:33.434156387
min,0 days 00:00:01
25%,0 days 00:00:30
50%,0 days 00:00:30
75%,0 days 00:01:00
99.95%,0 days 00:04:18
max,0 days 01:16:38


' \nThe statistic for Delta t shows that a large portion of the measurements have time differnce less than 3 minutes.\nBut it seems that at some point in our measurment process we did not record states for some nodes for around 2 hours.\nThis could our measurement faults or even the case that a specific nodes did not appear in the sinfo.\nWhat are those nodes? are they srv nodes?==> No idea\n'

In [10]:
"""  
How many times do we have this long pauses in the measurements?
"""
df_delta_counts = df.groupby(['node'], as_index=False)[['time']].diff().value_counts()
df_delta_counts.sort_index().tail(n=20)


time           
0 days 00:04:18    1548
0 days 00:04:21    1548
0 days 00:04:29    1548
0 days 00:04:48    1548
0 days 00:04:50    1548
0 days 00:05:02    1548
0 days 00:05:03    1548
0 days 00:05:05    1548
0 days 00:05:41    1548
0 days 00:05:57    1548
0 days 00:06:04    1548
0 days 00:06:29    1548
0 days 00:06:52    1548
0 days 00:06:54    1548
0 days 00:07:03    1548
0 days 00:08:35    1548
0 days 00:08:49    1548
0 days 00:09:53    1548
0 days 00:14:09    1548
0 days 01:16:38    1548
Name: count, dtype: int64

In [ ]:
# save the data a and go to EDA notebook for further analysis
# df.to_parquet(folder_path_slurm_data/'sinfo_cleaned.parquet.gzip', compression='gzip')